In [2]:
from torch import nn
import torch
import glob
import copy
import math
import random
import time
import math

In [33]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_input = 20
n_output = 2
n_hidden = 20
rnn = RNN(n_input, n_hidden, n_output)

# Read DataSet

In [4]:
def findFiles(path): return glob.glob(path)

filenames = findFiles('dataset/pretrain/*.csv')

In [5]:
def readCSV(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [line.split('\t') for line in lines]

def readCSVNumber(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [[int(num) for num in line.split('\t')] for line in lines]

In [6]:
datas = []
errdatas = []

for filename in filenames:
    try:
        datas.append(readCSVNumber(filename))
    except Exception as ex:
        errdatas.append(readCSV(filename))
        print(filename)
        print(ex)

dataset/pretrain/-newdisk-autofix-get-codeforce-src-utils-..-..-datas-1200-nimishkapoor-D-60043343OK.csv
invalid literal for int() with base 10: ''
dataset/pretrain/-newdisk-autofix-get-codeforce-src-utils-..-..-datas-1200-AbHiShEk2-B-61612132OK.csv
invalid literal for int() with base 10: ''
dataset/pretrain/-newdisk-autofix-get-codeforce-src-utils-..-..-datas-1200-dCoding-B-60583512OK.csv
invalid literal for int() with base 10: ''
dataset/pretrain/-newdisk-autofix-get-codeforce-src-utils-..-..-datas-1204-Alireza_Keshavarz-C-59172991OK.csv
invalid literal for int() with base 10: ''
dataset/pretrain/-newdisk-autofix-get-codeforce-src-utils-..-..-datas-1204-Chujinghan-C-59602008OK.csv
invalid literal for int() with base 10: ''


In [7]:
datas[0]

[[3, 1, 27, 3, 1, 27, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [3, 7, 33, 3, 7, 33, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [3, 17, 43, 3, 17, 43, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [7, 1, 72, 7, 1, 72, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [7, 7, 78, 7, 7, 78, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [7, 10, 81, 7, 10, 81, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [7, 12, 83, 7, 12, 83, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [7, 14, 85, 7, 14, 85, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [7, 18, 89, 7, 18, 89, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [7, 20, 91, 7, 20, 91, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [5, 13, 61, 5, 13, 61, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [5, 18, 66, 5, 18, 66, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [7, 1, 72, 7, 1, 72, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [7, 7, 78, 7, 7, 78, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [7, 10, 81, 7, 10, 81, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [7, 12

## 6 + 7 + 2 => DeclRefExpr
## 6 + 7 + 1 => VarDecl
## 6 + 7 + 3 => FunctionDecl
## 6 + 7 + 6 => ParmDecl

# Make Training Dataset

In [10]:
def changeData(data, declindex, refindex, changeNum):
    newdata = copy.deepcopy(data)
    changeList = []
    
    for i in range(changeNum):
        while True:
            idx = random.randint(0, len(refindex) - 1)
            
            # 선언된 변수가 1개거나 이미 바꾼 줄이면 다음 랜덤 인덱스
            if refindex[idx][1] == 1 or (idx in changeList):
                continue
            else:
                changeList.append(idx)

                while True:
                    decidx = random.randint(0, len(declindex) - 1)
                    # 기존의 변수랑 같은 거면 다른 변수
                    if newdata[declindex[decidx]][5] == newdata[refindex[idx][0]][2]:
                        continue
                    newdata[refindex[idx][0]][0] = newdata[declindex[decidx]][3]
                    newdata[refindex[idx][0]][1] = newdata[declindex[decidx]][4]
                    newdata[refindex[idx][0]][2] = newdata[declindex[decidx]][5]
                    break
                break
    
    return newdata, changeList

In [11]:
def makeTraining(dataset):
    trainData = []
    output = []
    
    for data in dataset:
        declindex = []
        refindex = []
        for idx, line in enumerate(data):
            if (line[14] == 1 or line[16] == 1 or line[19] == 1):
                declindex.append(idx)
            elif line[15] == 1:
                refindex.append([idx, len(declindex)])

        changeRate = 0.1
        changeNum = math.floor(changeRate * len(refindex))
        if changeNum > 0:
            for i in range(10):
                newdata, changeList = changeData(data, declindex, refindex, changeNum)
                trainData.append(newdata)
                output.append([0 if idx in changeList else 1 for idx in range(len(newdata))])
                
        print('done ', len(trainData), '/', len(dataset) * 10)
                
    return trainData, output

In [12]:
trainDataOrigin, outputOrigin = makeTraining(datas)

done  10 / 37550
done  20 / 37550
done  30 / 37550
done  40 / 37550
done  50 / 37550
done  60 / 37550
done  70 / 37550
done  80 / 37550
done  90 / 37550
done  100 / 37550
done  110 / 37550
done  120 / 37550
done  130 / 37550
done  140 / 37550
done  150 / 37550
done  160 / 37550
done  170 / 37550
done  180 / 37550
done  190 / 37550
done  200 / 37550
done  210 / 37550
done  220 / 37550
done  230 / 37550
done  240 / 37550
done  250 / 37550
done  260 / 37550
done  270 / 37550
done  280 / 37550
done  290 / 37550
done  300 / 37550
done  310 / 37550
done  320 / 37550
done  330 / 37550
done  340 / 37550
done  350 / 37550
done  360 / 37550
done  370 / 37550
done  380 / 37550
done  390 / 37550
done  400 / 37550
done  410 / 37550
done  420 / 37550
done  430 / 37550
done  440 / 37550
done  450 / 37550
done  460 / 37550
done  470 / 37550
done  480 / 37550
done  490 / 37550
done  500 / 37550
done  510 / 37550
done  520 / 37550
done  530 / 37550
done  540 / 37550
done  550 / 37550
done  560 / 37550
d

done  4430 / 37550
done  4440 / 37550
done  4450 / 37550
done  4460 / 37550
done  4470 / 37550
done  4480 / 37550
done  4490 / 37550
done  4500 / 37550
done  4510 / 37550
done  4520 / 37550
done  4530 / 37550
done  4540 / 37550
done  4550 / 37550
done  4560 / 37550
done  4570 / 37550
done  4580 / 37550
done  4590 / 37550
done  4600 / 37550
done  4610 / 37550
done  4620 / 37550
done  4630 / 37550
done  4640 / 37550
done  4650 / 37550
done  4660 / 37550
done  4670 / 37550
done  4680 / 37550
done  4690 / 37550
done  4700 / 37550
done  4710 / 37550
done  4720 / 37550
done  4730 / 37550
done  4740 / 37550
done  4750 / 37550
done  4760 / 37550
done  4770 / 37550
done  4780 / 37550
done  4790 / 37550
done  4800 / 37550
done  4810 / 37550
done  4820 / 37550
done  4830 / 37550
done  4840 / 37550
done  4850 / 37550
done  4860 / 37550
done  4870 / 37550
done  4880 / 37550
done  4890 / 37550
done  4900 / 37550
done  4910 / 37550
done  4920 / 37550
done  4930 / 37550
done  4940 / 37550
done  4950 /

done  8800 / 37550
done  8810 / 37550
done  8820 / 37550
done  8830 / 37550
done  8840 / 37550
done  8850 / 37550
done  8860 / 37550
done  8870 / 37550
done  8880 / 37550
done  8890 / 37550
done  8900 / 37550
done  8910 / 37550
done  8920 / 37550
done  8930 / 37550
done  8940 / 37550
done  8950 / 37550
done  8960 / 37550
done  8970 / 37550
done  8980 / 37550
done  8990 / 37550
done  9000 / 37550
done  9010 / 37550
done  9020 / 37550
done  9030 / 37550
done  9040 / 37550
done  9050 / 37550
done  9060 / 37550
done  9070 / 37550
done  9080 / 37550
done  9090 / 37550
done  9100 / 37550
done  9110 / 37550
done  9120 / 37550
done  9130 / 37550
done  9140 / 37550
done  9150 / 37550
done  9160 / 37550
done  9170 / 37550
done  9180 / 37550
done  9190 / 37550
done  9200 / 37550
done  9210 / 37550
done  9220 / 37550
done  9230 / 37550
done  9240 / 37550
done  9250 / 37550
done  9260 / 37550
done  9270 / 37550
done  9280 / 37550
done  9290 / 37550
done  9300 / 37550
done  9310 / 37550
done  9320 /

done  12980 / 37550
done  12990 / 37550
done  13000 / 37550
done  13010 / 37550
done  13020 / 37550
done  13030 / 37550
done  13040 / 37550
done  13050 / 37550
done  13060 / 37550
done  13070 / 37550
done  13080 / 37550
done  13090 / 37550
done  13100 / 37550
done  13110 / 37550
done  13120 / 37550
done  13130 / 37550
done  13140 / 37550
done  13150 / 37550
done  13160 / 37550
done  13170 / 37550
done  13180 / 37550
done  13190 / 37550
done  13200 / 37550
done  13210 / 37550
done  13220 / 37550
done  13230 / 37550
done  13240 / 37550
done  13250 / 37550
done  13260 / 37550
done  13270 / 37550
done  13280 / 37550
done  13290 / 37550
done  13300 / 37550
done  13310 / 37550
done  13320 / 37550
done  13330 / 37550
done  13340 / 37550
done  13350 / 37550
done  13360 / 37550
done  13370 / 37550
done  13380 / 37550
done  13390 / 37550
done  13400 / 37550
done  13410 / 37550
done  13420 / 37550
done  13430 / 37550
done  13440 / 37550
done  13450 / 37550
done  13460 / 37550
done  13470 / 37550


done  17090 / 37550
done  17100 / 37550
done  17110 / 37550
done  17120 / 37550
done  17130 / 37550
done  17140 / 37550
done  17150 / 37550
done  17160 / 37550
done  17170 / 37550
done  17180 / 37550
done  17190 / 37550
done  17200 / 37550
done  17210 / 37550
done  17220 / 37550
done  17230 / 37550
done  17240 / 37550
done  17250 / 37550
done  17260 / 37550
done  17270 / 37550
done  17280 / 37550
done  17290 / 37550
done  17300 / 37550
done  17310 / 37550
done  17320 / 37550
done  17330 / 37550
done  17340 / 37550
done  17350 / 37550
done  17360 / 37550
done  17370 / 37550
done  17380 / 37550
done  17390 / 37550
done  17400 / 37550
done  17410 / 37550
done  17420 / 37550
done  17430 / 37550
done  17440 / 37550
done  17450 / 37550
done  17460 / 37550
done  17470 / 37550
done  17480 / 37550
done  17490 / 37550
done  17500 / 37550
done  17510 / 37550
done  17520 / 37550
done  17530 / 37550
done  17540 / 37550
done  17550 / 37550
done  17560 / 37550
done  17570 / 37550
done  17580 / 37550


done  21240 / 37550
done  21250 / 37550
done  21260 / 37550
done  21270 / 37550
done  21280 / 37550
done  21290 / 37550
done  21300 / 37550
done  21310 / 37550
done  21320 / 37550
done  21330 / 37550
done  21340 / 37550
done  21350 / 37550
done  21360 / 37550
done  21370 / 37550
done  21380 / 37550
done  21390 / 37550
done  21400 / 37550
done  21410 / 37550
done  21420 / 37550
done  21430 / 37550
done  21440 / 37550
done  21450 / 37550
done  21460 / 37550
done  21470 / 37550
done  21480 / 37550
done  21490 / 37550
done  21500 / 37550
done  21510 / 37550
done  21520 / 37550
done  21530 / 37550
done  21540 / 37550
done  21550 / 37550
done  21560 / 37550
done  21570 / 37550
done  21580 / 37550
done  21590 / 37550
done  21600 / 37550
done  21610 / 37550
done  21620 / 37550
done  21630 / 37550
done  21640 / 37550
done  21650 / 37550
done  21660 / 37550
done  21670 / 37550
done  21680 / 37550
done  21690 / 37550
done  21700 / 37550
done  21710 / 37550
done  21720 / 37550
done  21730 / 37550


done  25350 / 37550
done  25360 / 37550
done  25370 / 37550
done  25380 / 37550
done  25390 / 37550
done  25400 / 37550
done  25410 / 37550
done  25420 / 37550
done  25430 / 37550
done  25440 / 37550
done  25450 / 37550
done  25460 / 37550
done  25470 / 37550
done  25480 / 37550
done  25490 / 37550
done  25500 / 37550
done  25510 / 37550
done  25520 / 37550
done  25530 / 37550
done  25540 / 37550
done  25550 / 37550
done  25560 / 37550
done  25570 / 37550
done  25580 / 37550
done  25590 / 37550
done  25600 / 37550
done  25610 / 37550
done  25620 / 37550
done  25630 / 37550
done  25640 / 37550
done  25650 / 37550
done  25660 / 37550
done  25670 / 37550
done  25680 / 37550
done  25690 / 37550
done  25700 / 37550
done  25710 / 37550
done  25720 / 37550
done  25730 / 37550
done  25740 / 37550
done  25750 / 37550
done  25760 / 37550
done  25770 / 37550
done  25780 / 37550
done  25790 / 37550
done  25800 / 37550
done  25810 / 37550
done  25820 / 37550
done  25830 / 37550
done  25840 / 37550


done  29450 / 37550
done  29460 / 37550
done  29470 / 37550
done  29480 / 37550
done  29490 / 37550
done  29500 / 37550
done  29510 / 37550
done  29520 / 37550
done  29530 / 37550
done  29540 / 37550
done  29550 / 37550
done  29560 / 37550
done  29570 / 37550
done  29580 / 37550
done  29590 / 37550
done  29600 / 37550
done  29610 / 37550
done  29620 / 37550
done  29630 / 37550
done  29640 / 37550
done  29650 / 37550
done  29660 / 37550
done  29670 / 37550
done  29680 / 37550
done  29690 / 37550
done  29700 / 37550
done  29710 / 37550
done  29720 / 37550
done  29730 / 37550
done  29740 / 37550
done  29750 / 37550
done  29760 / 37550
done  29770 / 37550
done  29780 / 37550
done  29790 / 37550
done  29800 / 37550
done  29810 / 37550
done  29820 / 37550
done  29830 / 37550
done  29840 / 37550
done  29850 / 37550
done  29860 / 37550
done  29870 / 37550
done  29880 / 37550
done  29890 / 37550
done  29900 / 37550
done  29910 / 37550
done  29920 / 37550
done  29930 / 37550
done  29940 / 37550


done  33600 / 37550
done  33610 / 37550
done  33620 / 37550
done  33630 / 37550
done  33640 / 37550
done  33650 / 37550
done  33660 / 37550
done  33670 / 37550
done  33680 / 37550
done  33690 / 37550
done  33700 / 37550
done  33710 / 37550
done  33720 / 37550
done  33730 / 37550
done  33740 / 37550
done  33750 / 37550
done  33760 / 37550
done  33770 / 37550
done  33780 / 37550
done  33790 / 37550
done  33800 / 37550
done  33810 / 37550
done  33820 / 37550
done  33830 / 37550
done  33840 / 37550
done  33850 / 37550
done  33860 / 37550
done  33870 / 37550
done  33880 / 37550
done  33890 / 37550
done  33900 / 37550
done  33910 / 37550
done  33920 / 37550
done  33930 / 37550
done  33940 / 37550
done  33950 / 37550
done  33960 / 37550
done  33970 / 37550
done  33980 / 37550
done  33990 / 37550
done  34000 / 37550
done  34010 / 37550
done  34020 / 37550
done  34030 / 37550
done  34040 / 37550
done  34050 / 37550
done  34060 / 37550
done  34070 / 37550
done  34080 / 37550
done  34090 / 37550


In [13]:
trainData = [torch.FloatTensor(data) for data in trainDataOrigin]
outputData = [torch.FloatTensor(data) for data in outputOrigin]

In [14]:
trainData = [data.reshape(len(data), 1, -1) for data in trainData]

outputData = [[data.reshape(1).long() for data in line] for line in outputData]

# Training

In [15]:
criterion = nn.NLLLoss()

In [35]:
learning_rate = 0.005

def train(tokenTensor, outputTensor):
    hidden = rnn.initHidden()
    
    for i in range(tokenTensor.size()[0]):
        rnn.zero_grad()
        output, hidden = rnn(tokenTensor[i], hidden)
        #print(output)
        #print(outputTensor[i].reshape(1, n_output).long())
        loss = criterion(output, outputTensor[i].reshape(1).long())
        loss.backward(retain_graph=True)

        for p in rnn.parameters():
            p.data.add_(-learning_rate, p.grad.data)
    
    return output, loss.item()

In [24]:
n_iters = 100000
print_every = 50
plot_every = 10

In [25]:
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

In [36]:
idx = 0
for iter in range(1, n_iters + 1):
    tokenTensor = trainData[idx]
    outputTensor = outputData[idx]
    output, loss = train(tokenTensor, outputTensor)
    current_loss += loss
    
    if iter % print_every == 0:
        print('%d %d%% (%s) %.4f' % (iter, iter / n_iters * 100, timeSince(start), loss))
    
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0
        
    print(iter)
    idx = (idx + 1) % len(trainData)

AttributeError: 'NoneType' object has no attribute 'data'

In [34]:
hidden = torch.zeros(1, n_hidden)
for i in range(trainData[0].size()[0]):
    result, hidden = rnn(trainData[0][i], hidden)
    print(result)

tensor([[-0.3825, -1.1462]], grad_fn=<LogSoftmaxBackward>)
tensor([[-0.8934, -0.5264]], grad_fn=<LogSoftmaxBackward>)
tensor([[-0.9412, -0.4945]], grad_fn=<LogSoftmaxBackward>)
tensor([[-1.4516, -0.2668]], grad_fn=<LogSoftmaxBackward>)
tensor([[-3.2065, -0.0413]], grad_fn=<LogSoftmaxBackward>)
tensor([[-3.9070, -0.0203]], grad_fn=<LogSoftmaxBackward>)
tensor([[-4.2513, -0.0143]], grad_fn=<LogSoftmaxBackward>)
tensor([[-4.0126, -0.0183]], grad_fn=<LogSoftmaxBackward>)
tensor([[-3.9968, -0.0185]], grad_fn=<LogSoftmaxBackward>)
tensor([[-4.3220, -0.0134]], grad_fn=<LogSoftmaxBackward>)
tensor([[-5.9177e+00, -2.6949e-03]], grad_fn=<LogSoftmaxBackward>)
tensor([[-4.1333, -0.0162]], grad_fn=<LogSoftmaxBackward>)
tensor([[-3.9531, -0.0194]], grad_fn=<LogSoftmaxBackward>)
tensor([[-4.2098, -0.0150]], grad_fn=<LogSoftmaxBackward>)
tensor([[-2.6385, -0.0742]], grad_fn=<LogSoftmaxBackward>)
tensor([[-4.0996, -0.0167]], grad_fn=<LogSoftmaxBackward>)
tensor([[-3.6328, -0.0268]], grad_fn=<LogSoftmax